# ADAU1401.Control Functional Test - with TCPi

https://www.analog.com/media/en/technical-documentation/data-sheets/adau1401.pdf

In [1]:
from pathfinder import Pathfinder

# abs_paths = [' ']
# Pathfinder.relative_paths_from_abs(abs_paths)
 
relative_paths = [['..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'SigmaDSP', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'Signal Generators', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
                  ['..', '..', 'codes']] 

Pathfinder.append_relative_paths(relative_paths)
# =====================================================

import pandas as pd

#https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)
# =====================================================

## TCPi

import time
import threading

from tcpi.protocols.TCPIP1701 import class_finder
# =====================================================

### Hardware bus

from bridges.ftdi.controllers.i2c import I2cController
from sigma.bus import adapters


with_hardware_device = True  # set True if hardware is connected.

if with_hardware_device:
    
    ctrl = I2cController() 
    _i2c = ctrl.I2C() 
    
else:
    _i2c = _pin_reset = None  # using None for testing without actual hardware device.

bus = adapters.I2C(_i2c)

# bus = fx2lp.I2C(as_400KHz = False)
# =====================================================

### Server

from tcpi.bus.tcpi_server import I2C as TcpI2C_server

tcpi_server = TcpI2C_server(bus, class_finder, i2c_addresses = {1: 0x68 >> 1, 2: 0xA0 >> 1})
print(tcpi_server.ip_address)
t_server = threading.Thread(target = tcpi_server.run)
t_server.start()
time.sleep(1)
# =====================================================

### Client

from tcpi.bus.tcpi_client import I2C as TcpI2C_client

tcpi_client = TcpI2C_client(class_finder)
tcpi_client.connect(*tcpi_server.ip_address)
time.sleep(1)
# =====================================================

## DSP processor

from sigma.sigma_dsp.adau import ADAU1401

# dsp = ADAU1401(bus = bus)
dsp = ADAU1401(bus = tcpi_client);
# =====================================================

('192.168.0.168', 8086)

[Server waiting for connection.]

[Connecting server: ('192.168.0.168', 8086)]

[Connected with server: ('192.168.0.168', 8086)]

[Connection from client ('192.168.0.168', 64410) established.]

[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 38, 0, 0]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2086
[ Data ]           :  [0, 0]
[ Data in Hex ]    :  ['0x0', '0x0']

[Server receiving: 156 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 34, 0, 0, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 16, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 24, 9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 28, 9, 1, 0

## Methods Test

### [Message](https://github.com/Wei1234c/SigmaDSP/blob/master/codes/sigma/sigma_dsp/messages.py) related methods

In [2]:
# load messages bytes from binary file

with open('E2Prom.bin', 'br') as f:
    messages_bytes = f.read()

In [3]:
# parse data bytes in to a collections of "Message" object.

messages = dsp.control.messages_from_bytes(messages_bytes)
for m in messages:
    print(m)

{'Type': 'Write', 'Type ID': 1, 'Device Address': 0, 'Sub Address': '0x081C', 'Data': b'\x00X'}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'Write', 'Type ID': 1, 'Device 

In [4]:
# re-write the whole configuration (program, parameters, control registers), equvalient to software reset.

dsp.control.reload_from_eeprom()


[Server receiving: 8 bytes of data.]
[10, 0, 8, 80, 1, 244, 0, 0]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  0

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  0
[ Success ]      :  0
[ Data ]         :  [1, 0, 5, 0, 8, 28, 0, 88, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 0, 35, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 235, 0, 0, 0, 0, 0, 0, 255, 0, 0, 150, 48, 0, 128, 0, 0, 0, 0, 0, 255, 0, 0, 155, 166, 0, 128, 0, 0, 0, 128, 0, 0, 0, 5, 70, 75, 0, 0, 0, 0, 0, 0, 0, 2, 0, 128, 0, 0, 0, 0, 8, 0, 0, 128, 0, 0, 0, 0, 8, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 55, 92, 15, 0, 144, 116, 0, 127, 58, 95, 0, 255, 111, 140, 15, 128, 142, 68, 0, 128, 55, 92, 15, 0, 144, 116, 0, 127, 58, 95, 0, 255, 111, 140, 15, 


[Server receiving: 8 bytes of data.]
[10, 0, 8, 80, 1, 244, 3, 232]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  1000

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  1000
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 90, 130, 122, 0, 64, 0, 0, 0, 45, 65, 61, 0, 32, 0, 0, 0, 22, 160, 158, 0, 16, 0, 0, 0, 0, 19, 7, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 81, 236, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  2000
[ Success ]      :  0
[ Data ]         :  [226, 1, 255, 233, 8, 32, 1, 1, 240, 0, 226, 1, 1, 145, 8, 32, 1, 1, 153, 8, 34, 73, 255, 241, 8, 32, 38, 1, 240, 0, 226, 1, 1, 177, 8, 32, 1, 255, 241, 19, 34, 73, 1, 145, 8, 32, 1, 1, 209, 8, 32, 37, 1, 216, 0, 226, 1, 1, 193, 8, 32, 1, 255, 241, 19, 34, 73, 1, 145, 8, 32, 1, 1, 225, 8, 32, 37, 1, 232, 0, 226, 1, 1, 249, 8, 32, 1, 0, 0, 0, 0, 1, 2, 8, 0, 240, 1, 2, 9, 8, 32, 1, 255, 241, 19, 34, 73, 1, 241, 8, 32, 1, 255, 233, 8, 32, 37, 1, 240, 0, 226, 1, 1, 145, 8, 32, 1, 1, 152, 0, 226, 37, 1, 161, 8, 32, 1, 1, 177, 8, 34, 65, 2, 0, 0, 226, 1, 1, 177, 8, 32, 1, 2, 1, 20, 34, 1, 1, 184, 0, 226, 1, 1, 216, 0, 192, 1, 0, 7, 255, 160, 1, 1, 184, 0, 192, 1, 1, 103, 255, 32, 129, 1, 80, 0, 226, 1, 1, 127, 255, 64, 129, 1, 88, 0, 228, 1, 255, 241, 8, 32, 1, 255, 247, 255, 34, 65, 1, 200, 0, 226, 1, 1, 232, 0,


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  3500
[ Success ]      :  0
[ Data ]         :  [226, 129, 0, 2, 124, 160, 1, 6, 249, 8, 32, 1, 6, 240, 0, 226, 1, 6, 97, 8, 32, 1, 6, 248, 0, 226, 129, 0, 2, 125, 160, 1, 7, 17, 8, 32, 1, 7, 8, 0, 226, 1, 6, 185, 8, 32, 1, 7, 16, 0, 226, 129, 6, 194, 126, 32, 1, 7, 32, 0, 226, 1, 0, 0, 0, 0, 1, 6, 218, 127, 32, 1, 7, 40, 0, 226, 1, 0, 0, 0, 0, 1, 6, 242, 128, 32, 1, 7, 48, 0, 226, 1, 0, 0, 0, 0, 1, 7, 10, 129, 32, 1, 7, 56, 0, 226, 1, 0, 0, 0, 0, 1, 255, 242, 130, 32, 1, 7, 81, 8, 34, 65, 7, 112, 0, 226, 1, 7, 97, 8, 32, 1, 7, 81, 8, 52, 1, 7, 114, 131, 34, 1, 7, 88, 0, 226, 1, 7, 88, 0, 192, 1, 7, 104, 0, 242, 1, 7, 39, 255, 32, 1, 7, 64, 0, 226, 1, 7, 47, 255, 32, 1, 7, 72, 0, 226, 1, 255, 242, 132, 32, 1, 7, 137, 8, 34, 65, 7, 168, 0, 226, 1, 7, 153, 8, 32, 1, 7, 137, 8, 52, 1, 7, 170, 133, 34, 1, 7, 144, 0, 226, 1, 7, 144, 0, 192, 1, 7, 160, 0, 242,


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  5000
[ Success ]      :  0
[ Data ]         :  [0, 1, 0, 3, 11, 161, 1, 255, 225, 8, 32, 1, 0, 3, 10, 161, 1, 255, 225, 8, 34, 65, 9, 240, 0, 226, 1, 10, 40, 0, 192, 1, 9, 247, 255, 32, 1, 255, 225, 8, 34, 1, 10, 0, 0, 226, 1, 10, 0, 0, 192, 1, 7, 127, 255, 32, 1, 9, 168, 0, 226, 1, 7, 179, 54, 32, 1, 10, 80, 0, 226, 1, 8, 91, 55, 32, 1, 10, 88, 0, 226, 1, 9, 171, 56, 32, 1, 10, 96, 0, 226, 1, 9, 3, 57, 32, 1, 10, 104, 0, 226, 1, 10, 81, 8, 32, 1, 255, 104, 0, 2, 1, 10, 89, 8, 32, 1, 255, 112, 0, 2, 1, 10, 97, 8, 32, 1, 255, 120, 0, 2, 1, 10, 105, 8, 32, 1, 255, 128, 0, 2, 1, 1, 0, 5, 0, 8, 28, 0, 28, 1, 0, 4, 0, 8, 29, 8, 1, 0, 5, 0, 8, 30, 0, 0, 1, 0, 4, 0, 8, 31, 0, 1, 0, 6, 0, 8, 32, 0, 0, 0, 1, 0, 6, 0, 8, 33, 0, 0, 0, 1, 0, 5, 0, 8, 34, 0, 0, 1, 0, 5, 0, 8, 35, 0, 0, 1, 0, 5, 0, 8, 36, 128, 0, 1, 0, 5, 0, 8, 37, 0, 0, 1, 0, 5, 0, 8, 38, 0, 0, 1, 0

...


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  10000
[ Success ]      :  0
[ Data ]         :  [248, 0, 226, 1, 1, 24, 0, 192, 1, 0, 7, 255, 160, 1, 0, 248, 0, 192, 1, 0, 167, 255, 32, 129, 0, 144, 0, 226, 1, 0, 191, 255, 64, 129, 0, 152, 0, 228, 1, 255, 241, 8, 32, 1, 255, 247, 255, 34, 65, 1, 8, 0, 226, 1, 1, 40, 0, 192, 1, 0, 7, 255, 160, 1, 1, 8, 0, 192, 1, 0, 167, 255, 32, 129, 0, 145, 8, 34, 1, 0, 144, 0, 226, 1, 0, 191, 255, 64, 129, 0, 153, 8, 68, 1, 0, 152, 0, 228, 1, 0, 225, 8, 32, 1, 1, 49, 8, 34, 65, 0, 0, 0, 0, 1, 0, 232, 0, 240, 1, 0, 0, 0, 0, 1, 255, 242, 9, 32, 1, 1, 152, 0, 226, 1, 0, 1, 8, 32, 1, 1, 96, 0, 226, 1, 0, 65, 8, 32, 1, 1, 120, 0, 226, 1, 0, 121, 8, 32, 1, 1, 104, 0, 226, 1, 0, 121, 8, 32, 1, 1, 128, 0, 226, 1, 0, 105, 8, 32, 1, 1, 112, 0, 226, 1, 0, 113, 8, 32, 1, 1, 136, 0, 226, 1, 255, 233, 8, 32, 1, 1, 240, 0, 226, 1, 1, 145, 8, 32, 1, 1, 153, 8, 34, 73, 255, 241, 8,


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  11500
[ Success ]      :  0
[ Data ]         :  [250, 93, 34, 1, 6, 2, 92, 34, 1, 6, 8, 0, 226, 1, 4, 249, 8, 32, 1, 6, 88, 0, 226, 1, 6, 18, 111, 32, 1, 6, 26, 110, 34, 1, 6, 34, 109, 34, 1, 6, 42, 108, 34, 1, 6, 50, 107, 34, 1, 6, 58, 106, 34, 1, 6, 66, 105, 34, 1, 6, 74, 104, 34, 1, 6, 82, 103, 34, 1, 6, 90, 102, 34, 1, 6, 96, 0, 226, 1, 5, 65, 8, 32, 1, 6, 176, 0, 226, 1, 6, 106, 121, 32, 1, 6, 114, 120, 34, 1, 6, 122, 119, 34, 1, 6, 130, 118, 34, 1, 6, 138, 117, 34, 1, 6, 146, 116, 34, 1, 6, 154, 115, 34, 1, 6, 162, 114, 34, 1, 6, 170, 113, 34, 1, 6, 178, 112, 34, 1, 6, 184, 0, 226, 1, 0, 2, 122, 160, 1, 6, 201, 8, 32, 1, 6, 192, 0, 226, 1, 5, 177, 8, 32, 1, 6, 200, 0, 226, 129, 0, 2, 123, 160, 1, 6, 225, 8, 32, 1, 6, 216, 0, 226, 1, 6, 9, 8, 32, 1, 6, 224, 0, 226, 129, 0, 2, 124, 160, 1, 6, 249, 8, 32, 1, 6, 240, 0, 226, 1, 6, 97, 8, 32, 1, 6, 248


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  13000
[ Success ]      :  0
[ Data ]         :  [193, 8, 32, 1, 9, 195, 50, 34, 65, 10, 19, 50, 34, 1, 9, 177, 8, 52, 1, 9, 179, 50, 34, 65, 10, 11, 50, 34, 1, 9, 184, 0, 226, 1, 255, 233, 8, 66, 1, 9, 200, 0, 242, 1, 10, 65, 8, 32, 1, 255, 233, 8, 130, 1, 255, 233, 8, 132, 35, 255, 241, 10, 32, 1, 0, 0, 0, 0, 1, 10, 72, 0, 246, 1, 10, 73, 8, 34, 1, 10, 24, 0, 226, 1, 0, 0, 0, 0, 1, 10, 24, 0, 246, 35, 9, 225, 8, 32, 9, 9, 232, 0, 226, 1, 9, 209, 8, 32, 1, 255, 243, 52, 34, 103, 9, 216, 0, 226, 1, 10, 25, 8, 34, 73, 10, 25, 8, 32, 1, 9, 216, 0, 226, 39, 255, 243, 51, 32, 1, 9, 232, 0, 226, 39, 0, 0, 0, 0, 1, 9, 233, 8, 32, 9, 255, 249, 8, 34, 65, 9, 232, 0, 226, 38, 9, 217, 26, 32, 1, 0, 0, 0, 0, 1, 10, 32, 0, 226, 1, 10, 40, 0, 242, 1, 10, 32, 0, 192, 1, 0, 0, 0, 0, 1, 0, 3, 11, 161, 1, 255, 225, 8, 32, 1, 0, 3, 10, 161, 1, 255, 225, 8, 34, 65, 9, 240,


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  14500
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  393
[ Chip_address ] :  80
[ Data_length ]  :  384
[ Address ]      :  16000
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [5]:
#  write a "Message" to hardware.
 
dsp.control.write_message(messages[0])
print(messages[0])

{'Type': 'Write', 'Type ID': 1, 'Device Address': 0, 'Sub Address': '0x081C', 'Data': b'\x00X'}


### XML related methods

In [6]:
import os
from sigma.sigma_studio.project.project_xml import *

project_xml_file_url = os.sep.join(['..', '..', 'SigmaStudio projects', 'projects', 'demo', 'demo.xml']) 
ic = get_ICs(project_xml_file_url)[0]


[Server receiving: 4392 bytes of data.]
[9, 1, 0, 0, 70, 52, 0, 60, 4, 12, 0, 40, 0, 226, 1, 0, 32, 0, 192, 1, 0, 47, 255, 32, 1, 255, 225, 8, 34, 1, 0, 48, 0, 226, 1, 0, 50, 2, 32, 1, 0, 0, 0, 226, 1, 0, 89, 31, 32, 1, 0, 73, 8, 34, 1, 0, 2, 4, 160, 1, 255, 225, 24, 34, 1, 0, 2, 3, 160, 1, 9, 1, 0, 0, 70, 52, 0, 60, 4, 24, 0, 80, 0, 234, 1, 0, 96, 0, 242, 1, 0, 80, 0, 192, 1, 0, 0, 0, 0, 1, 255, 240, 1, 33, 1, 0, 0, 0, 161, 1, 255, 225, 8, 34, 65, 0, 40, 0, 226, 1, 0, 96, 0, 192, 1, 0, 47, 255, 32, 1, 255, 225, 8, 34, 1, 0, 48, 0, 226, 1, 9, 1, 0, 0, 70, 52, 0, 60, 4, 36, 0, 50, 5, 32, 1, 0, 64, 0, 226, 1, 0, 0, 0, 0, 1, 0, 104, 0, 232, 1, 0, 0, 0, 0, 1, 0, 112, 0, 232, 1, 255, 242, 6, 32, 9, 0, 130, 7, 32, 1, 255, 249, 31, 34, 1, 0, 136, 0, 242, 1, 255, 241, 9, 50, 65, 255, 233, 8, 32, 39, 9, 1, 0, 0, 70, 52, 0, 60, 4, 48, 0, 120, 0, 226, 1, 0, 0, 0, 0, 1, 255, 242, 8, 32, 1, 0, 216, 0, 226, 1, 0, 1, 8, 32, 1, 0, 160, 0, 226, 1, 0, 65, 8, 32, 1, 0, 184, 0, 226, 1, 0, 121, 8, 32, 1, 

[ Chip_address ]   :  52
[ Data_length ]    :  60
[ Address ]        :  1036
[ Data ]           :  [0, 40, 0, 226, 1, 0, 32, 0, 192, 1, 0, 47, 255, 32, 1, 255, 225, 8, 34, 1, 0, 48, 0, 226, 1, 0, 50, 2, 32, 1, 0, 0, 0, 226, 1, 0, 89, 31, 32, 1, 0, 73, 8, 34, 1, 0, 2, 4, 160, 1, 255, 225, 24, 34, 1, 0, 2, 3, 160, 1]
[ Data in Hex ]    :  ['0x0', '0x28', '0x0', '0xe2', '0x1', '0x0', '0x20', '0x0', '0xc0', '0x1', '0x0', '0x2f', '0xff', '0x20', '0x1', '0xff', '0xe1', '0x8', '0x22', '0x1', '0x0', '0x30', '0x0', '0xe2', '0x1', '0x0', '0x32', '0x2', '0x20', '0x1', '0x0', '0x0', '0x0', '0xe2', '0x1', '0x0', '0x59', '0x1f', '0x20', '0x1', '0x0', '0x49', '0x8', '0x22', '0x1', '0x0', '0x2', '0x4', '0xa0', '0x1', '0xff', '0xe1', '0x18', '0x22', '0x1', '0x0', '0x2', '0x3', '0xa0', '0x1']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  70
[ Chip_address ]   :  52
[ Data_length ]    :  60
[ Address ]        :  1048
[ Data ]         


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  70
[ Chip_address ]   :  52
[ Data_length ]    :  60
[ Address ]        :  1168
[ Data ]           :  [1, 216, 0, 226, 1, 1, 193, 8, 32, 1, 255, 241, 19, 34, 73, 1, 145, 8, 32, 1, 1, 225, 8, 32, 37, 1, 232, 0, 226, 1, 1, 249, 8, 32, 1, 0, 0, 0, 0, 1, 2, 8, 0, 240, 1, 2, 9, 8, 32, 1, 255, 241, 19, 34, 73, 1, 241, 8, 32, 1]
[ Data in Hex ]    :  ['0x1', '0xd8', '0x0', '0xe2', '0x1', '0x1', '0xc1', '0x8', '0x20', '0x1', '0xff', '0xf1', '0x13', '0x22', '0x49', '0x1', '0x91', '0x8', '0x20', '0x1', '0x1', '0xe1', '0x8', '0x20', '0x25', '0x1', '0xe8', '0x0', '0xe2', '0x1', '0x1', '0xf9', '0x8', '0x20', '0x1', '0x0', '0x0', '0x0', '0x0', '0x1', '0x2', '0x8', '0x0', '0xf0', '0x1', '0x2', '0x9', '0x8', '0x20', '0x1', '0xff', '0xf1', '0x13', '0x22', '0x49', '0x1', '0xf1', '0x8', '0x20', '0x1']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ]


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  70
[ Chip_address ]   :  52
[ Data_length ]    :  60
[ Address ]        :  1300
[ Data ]           :  [3, 162, 42, 32, 1, 3, 192, 0, 226, 1, 3, 170, 43, 32, 1, 3, 200, 0, 226, 1, 3, 162, 44, 32, 1, 3, 208, 0, 226, 1, 3, 162, 45, 32, 1, 3, 216, 0, 226, 1, 3, 162, 46, 32, 1, 3, 224, 0, 226, 1, 3, 170, 47, 32, 1, 3, 232, 0, 226, 1]
[ Data in Hex ]    :  ['0x3', '0xa2', '0x2a', '0x20', '0x1', '0x3', '0xc0', '0x0', '0xe2', '0x1', '0x3', '0xaa', '0x2b', '0x20', '0x1', '0x3', '0xc8', '0x0', '0xe2', '0x1', '0x3', '0xa2', '0x2c', '0x20', '0x1', '0x3', '0xd0', '0x0', '0xe2', '0x1', '0x3', '0xa2', '0x2d', '0x20', '0x1', '0x3', '0xd8', '0x0', '0xe2', '0x1', '0x3', '0xa2', '0x2e', '0x20', '0x1', '0x3', '0xe0', '0x0', '0xe2', '0x1', '0x3', '0xaa', '0x2f', '0x20', '0x1', '0x3', '0xe8', '0x0', '0xe2', '0x1']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channe

[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  70
[ Chip_address ]   :  52
[ Data_length ]    :  60
[ Address ]        :  1408
[ Data ]           :  [5, 176, 0, 226, 1, 4, 177, 8, 32, 1, 6, 0, 0, 226, 1, 5, 186, 101, 32, 1, 5, 194, 100, 34, 1, 5, 202, 99, 34, 1, 5, 210, 98, 34, 1, 5, 218, 97, 34, 1, 5, 226, 96, 34, 1, 5, 234, 95, 34, 1, 5, 242, 94, 34, 1, 5, 250, 93, 34, 1]
[ Data in Hex ]    :  ['0x5', '0xb0', '0x0', '0xe2', '0x1', '0x4', '0xb1', '0x8', '0x20', '0x1', '0x6', '0x0', '0x0', '0xe2', '0x1', '0x5', '0xba', '0x65', '0x20', '0x1', '0x5', '0xc2', '0x64', '0x22', '0x1', '0x5', '0xca', '0x63', '0x22', '0x1', '0x5', '0xd2', '0x62', '0x22', '0x1', '0x5', '0xda', '0x61', '0x22', '0x1', '0x5', '0xe2', '0x60', '0x22', '0x1', '0x5', '0xea', '0x5f', '0x22', '0x1', '0x5', '0xf2', '0x5e', '0x22', '0x1', '0x5', '0xfa', '0x5d', '0x22', '0x1']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Tota


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  70
[ Chip_address ]   :  52
[ Data_length ]    :  60
[ Address ]        :  1552
[ Data ]           :  [255, 249, 8, 34, 65, 7, 240, 0, 226, 38, 7, 225, 26, 32, 1, 0, 0, 0, 0, 1, 8, 40, 0, 226, 1, 8, 48, 0, 242, 1, 8, 40, 0, 192, 1, 0, 0, 0, 0, 1, 0, 2, 135, 161, 1, 255, 225, 8, 32, 1, 0, 2, 134, 161, 1, 255, 225, 8, 34, 65]
[ Data in Hex ]    :  ['0xff', '0xf9', '0x8', '0x22', '0x41', '0x7', '0xf0', '0x0', '0xe2', '0x26', '0x7', '0xe1', '0x1a', '0x20', '0x1', '0x0', '0x0', '0x0', '0x0', '0x1', '0x8', '0x28', '0x0', '0xe2', '0x1', '0x8', '0x30', '0x0', '0xf2', '0x1', '0x8', '0x28', '0x0', '0xc0', '0x1', '0x0', '0x0', '0x0', '0x0', '0x1', '0x0', '0x2', '0x87', '0xa1', '0x1', '0xff', '0xe1', '0x8', '0x20', '0x1', '0x0', '0x2', '0x86', '0xa1', '0x1', '0xff', '0xe1', '0x8', '0x22', '0x41']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  70
[ Chip_address ]   :  52
[ Data_length ]    :  60
[ Address ]        :  1660
[ Data ]           :  [255, 233, 8, 66, 1, 9, 32, 0, 242, 1, 9, 153, 8, 32, 1, 255, 233, 8, 130, 1, 255, 233, 8, 132, 35, 255, 241, 10, 32, 1, 0, 0, 0, 0, 1, 9, 160, 0, 246, 1, 9, 161, 8, 34, 1, 9, 112, 0, 226, 1, 0, 0, 0, 0, 1, 9, 112, 0, 246, 35]
[ Data in Hex ]    :  ['0xff', '0xe9', '0x8', '0x42', '0x1', '0x9', '0x20', '0x0', '0xf2', '0x1', '0x9', '0x99', '0x8', '0x20', '0x1', '0xff', '0xe9', '0x8', '0x82', '0x1', '0xff', '0xe9', '0x8', '0x84', '0x23', '0xff', '0xf1', '0xa', '0x20', '0x1', '0x0', '0x0', '0x0', '0x0', '0x1', '0x9', '0xa0', '0x0', '0xf6', '0x1', '0x9', '0xa1', '0x8', '0x22', '0x1', '0x9', '0x70', '0x0', '0xe2', '0x1', '0x0', '0x0', '0x0', '0x0', '0x1', '0x9', '0x70', '0x0', '0xf6', '0x23']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_numb

In [7]:
# read SigmaStudio project XML file and re-write the whole configuration (program, parameters, control registers).

dsp.control.write_xml_ic(ic)


[Server receiving: 5130 bytes of data.]
[9, 1, 0, 20, 10, 52, 20, 0, 4, 0, 0, 25, 31, 32, 1, 0, 9, 8, 34, 1, 0, 2, 1, 160, 1, 255, 225, 24, 34, 1, 0, 2, 0, 160, 1, 0, 16, 0, 234, 1, 0, 32, 0, 242, 1, 0, 16, 0, 192, 1, 0, 0, 0, 0, 1, 255, 240, 1, 33, 1, 0, 0, 0, 161, 1, 255, 225, 8, 34, 65, 0, 40, 0, 226, 1, 0, 32, 0, 192, 1, 0, 47, 255, 32, 1, 255, 225, 8, 34, 1, 0, 48, 0, 226, 1, 0, 50, 2, 32, 1, 0, 0, 0, 226, 1, 0, 89, 31, 32, 1, 0, 73, 8, 34, 1, 0, 2, 4, 160, 1, 255, 225, 24, 34, 1, 0, 2, 3, 160, 1, 0, 80, 0, 234, 1, 0, 96, 0, 242, 1, 0, 80, 0, 192, 1, 0, 0, 0, 0, 1, 255, 240, 1, 33, 1, 0, 0, 0, 161, 1, 255, 225, 8, 34, 65, 0, 40, 0, 226, 1, 0, 96, 0, 192, 1, 0, 47, 255, 32, 1, 255, 225, 8, 34, 1, 0, 48, 0, 226, 1, 0, 50, 5, 32, 1, 0, 64, 0, 226, 1, 0, 0, 0, 0, 1, 0, 104, 0, 232, 1, 0, 0, 0, 0, 1, 0, 112, 0, 232, 1, 255, 242, 6, 32, 9, 0, 130, 7, 32, 1, 255, 249, 31, 34, 1, 0, 136, 0, 242, 1, 255, 241, 9, 50, 65, 255, 233, 8, 32, 39, 0, 120, 0, 226, 1, 0, 0, 0, 0, 1, 255, 242, 8, 3


[Server receiving: 1324 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 24, 9, 1, 0, 4, 242, 52, 4, 232, 0, 0, 0, 0, 0, 255, 0, 0, 150, 48, 0, 128, 0, 0, 0, 0, 0, 255, 0, 0, 155, 166, 0, 128, 0, 0, 0, 128, 0, 0, 0, 5, 70, 75, 0, 0, 0, 0, 0, 0, 0, 2, 0, 128, 0, 0, 0, 0, 8, 0, 0, 128, 0, 0, 0, 0, 8, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 55, 91, 15, 0, 144, 117, 0, 127, 58, 94, 0, 255, 111, 139, 15, 128, 142, 68, 0, 128, 55, 91, 15, 0, 144, 117, 0, 127, 58, 94, 0, 255, 111, 139, 15, 128, 142, 68, 0, 128, 55, 91, 15, 0, 144, 117, 0, 127, 58, 94, 0, 255, 111, 139, 15, 128, 142, 68, 0, 128, 55, 91, 15, 0, 144, 117, 0, 127, 58, 94, 0, 255, 111, 139, 15, 128, 142, 68, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 0, 0, 0, 32, 0, 0, 0, 32, 0, 0, 0, 32, 0, 0, 0


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  34
[ Chip_address ]   :  52
[ Data_length ]    :  24
[ Address ]        :  2076
[ Data ]           :  [0, 24, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 0, 0, 0, 0, 0, 0, 1]
[ Data in Hex ]    :  ['0x0', '0x18', '0x8', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x1']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2076
[ Data ]           :  [0, 28]
[ Data in Hex ]    :  ['0x0', '0x1c']


### Text file related methods

In [9]:
# load SigmaStudio files and re-write the whole configuration (program, parameters, control registers).

dsp.control.load_SigmaStudio_files(file_NumBytes = 'NumBytes_IC_1.dat', file_TxBuffer = 'TxBuffer_IC_1.dat')


[Server receiving: 12 bytes of data.]
[9, 1, 0, 0, 12, 52, 0, 2, 8, 28, 0, 24]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2076
[ Data ]           :  [0, 24]
[ Data in Hex ]    :  ['0x0', '0x18']

[Server receiving: 9282 bytes of data.]
[9, 1, 0, 20, 10, 52, 20, 0, 4, 0, 0, 25, 31, 32, 1, 0, 9, 8, 34, 1, 0, 2, 1, 160, 1, 255, 225, 24, 34, 1, 0, 2, 0, 160, 1, 0, 16, 0, 234, 1, 0, 32, 0, 242, 1, 0, 16, 0, 192, 1, 0, 0, 0, 0, 1, 255, 240, 1, 33, 1, 0, 0, 0, 161, 1, 255, 225, 8, 34, 65, 0, 40, 0, 226, 1, 0, 32, 0, 192, 1, 0, 47, 255, 32, 1, 255, 225, 8, 34, 1, 0, 48, 0, 226, 1, 0, 50, 2, 32, 1, 0, 0, 0, 226, 1, 0, 89, 31, 32, 1, 0, 73, 8, 34, 1, 0, 2, 4, 160, 1, 255, 225, 24, 34, 1, 0, 2, 3, 160, 1, 0, 80, 0, 234, 1, 0, 96, 0, 242, 1, 0, 80, 0, 192, 1, 0, 0, 0, 0, 1, 255, 240, 1, 33, 1, 0, 0, 0, 161, 1, 255, 225, 8, 34, 65, 0, 40, 0, 226, 1, 0,


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  4106
[ Chip_address ]   :  52
[ Data_length ]    :  4096
[ Address ]        :  0
[ Data ]           :  [0, 0, 0, 255, 0, 0, 150, 48, 0, 128, 0, 0, 0, 0, 0, 255, 0, 0, 155, 166, 0, 128, 0, 0, 0, 128, 0, 0, 0, 5, 70, 75, 0, 0, 0, 0, 0, 0, 0, 2, 0, 128, 0, 0, 0, 0, 8, 0, 0, 128, 0, 0, 0, 0, 8, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 55, 92, 15, 0, 144, 116, 0, 127, 58, 95, 0, 255, 111, 140, 15, 128, 142, 68, 0, 128, 55, 92, 15, 0, 144, 116, 0, 127, 58, 95, 0, 255, 111, 140, 15, 128, 142, 68, 0, 128, 55, 92, 15, 0, 144, 116, 0, 127, 58, 95, 0, 255, 111, 140, 15, 128, 142, 68, 0, 128, 55, 92, 15, 0, 144, 116, 0, 127, 58, 95, 0, 255, 111, 140, 15, 128, 142, 68, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  34
[ Chip_address ]   :  52
[ Data_length ]    :  24
[ Address ]        :  2076
[ Data ]           :  [0, 24, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 0, 0, 0, 0, 0, 0, 1]
[ Data in Hex ]    :  ['0x0', '0x18', '0x8', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x1']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  12
[ Chip_address ]   :  52
[ Data_length ]    :  2
[ Address ]        :  2076
[ Data ]           :  [0, 28]
[ Data in Hex ]    :  ['0x0', '0x1c']


### EEPROM related methods

In [10]:
# dump_eeprom_to_file: dump data of EEPROM to a binary file.

dsp.control.dump_eeprom_to_file(binary_file_name = 'eeprom.bin')


[Server receiving: 8 bytes of data.]
[10, 0, 8, 80, 1, 244, 0, 0]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  0

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  0
[ Success ]      :  0
[ Data ]         :  [1, 0, 5, 0, 8, 28, 0, 88, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 0, 35, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 235, 0, 0, 0, 0, 0, 0, 255, 0, 0, 150, 48, 0, 128, 0, 0, 0, 0, 0, 255, 0, 0, 155, 166, 0, 128, 0, 0, 0, 128, 0, 0, 0, 5, 70, 75, 0, 0, 0, 0, 0, 0, 0, 2, 0, 128, 0, 0, 0, 0, 8, 0, 0, 128, 0, 0, 0, 0, 8, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 55, 92, 15, 0, 144, 116, 0, 127, 58, 95, 0, 255, 111, 140, 15, 128, 142, 68, 0, 128, 55, 92, 15, 0, 144, 116, 0, 127, 58, 95, 0, 255, 111, 140, 15, 


[Server receiving: 8 bytes of data.]
[10, 0, 8, 80, 1, 244, 3, 232]

<< ReadRequest >>  
[ Control ]      :  10
[ Total_length ] :  8
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  1000

<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  1000
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 90, 130, 122, 0, 64, 0, 0, 0, 45, 65, 61, 0, 32, 0, 0, 0, 22, 160, 158, 0, 16, 0, 0, 0, 0, 19, 7, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 81, 236, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  2000
[ Success ]      :  0
[ Data ]         :  [226, 1, 255, 233, 8, 32, 1, 1, 240, 0, 226, 1, 1, 145, 8, 32, 1, 1, 153, 8, 34, 73, 255, 241, 8, 32, 38, 1, 240, 0, 226, 1, 1, 177, 8, 32, 1, 255, 241, 19, 34, 73, 1, 145, 8, 32, 1, 1, 209, 8, 32, 37, 1, 216, 0, 226, 1, 1, 193, 8, 32, 1, 255, 241, 19, 34, 73, 1, 145, 8, 32, 1, 1, 225, 8, 32, 37, 1, 232, 0, 226, 1, 1, 249, 8, 32, 1, 0, 0, 0, 0, 1, 2, 8, 0, 240, 1, 2, 9, 8, 32, 1, 255, 241, 19, 34, 73, 1, 241, 8, 32, 1, 255, 233, 8, 32, 37, 1, 240, 0, 226, 1, 1, 145, 8, 32, 1, 1, 152, 0, 226, 37, 1, 161, 8, 32, 1, 1, 177, 8, 34, 65, 2, 0, 0, 226, 1, 1, 177, 8, 32, 1, 2, 1, 20, 34, 1, 1, 184, 0, 226, 1, 1, 216, 0, 192, 1, 0, 7, 255, 160, 1, 1, 184, 0, 192, 1, 1, 103, 255, 32, 129, 1, 80, 0, 226, 1, 1, 127, 255, 64, 129, 1, 88, 0, 228, 1, 255, 241, 8, 32, 1, 255, 247, 255, 34, 65, 1, 200, 0, 226, 1, 1, 232, 0,


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  3500
[ Success ]      :  0
[ Data ]         :  [226, 129, 0, 2, 124, 160, 1, 6, 249, 8, 32, 1, 6, 240, 0, 226, 1, 6, 97, 8, 32, 1, 6, 248, 0, 226, 129, 0, 2, 125, 160, 1, 7, 17, 8, 32, 1, 7, 8, 0, 226, 1, 6, 185, 8, 32, 1, 7, 16, 0, 226, 129, 6, 194, 126, 32, 1, 7, 32, 0, 226, 1, 0, 0, 0, 0, 1, 6, 218, 127, 32, 1, 7, 40, 0, 226, 1, 0, 0, 0, 0, 1, 6, 242, 128, 32, 1, 7, 48, 0, 226, 1, 0, 0, 0, 0, 1, 7, 10, 129, 32, 1, 7, 56, 0, 226, 1, 0, 0, 0, 0, 1, 255, 242, 130, 32, 1, 7, 81, 8, 34, 65, 7, 112, 0, 226, 1, 7, 97, 8, 32, 1, 7, 81, 8, 52, 1, 7, 114, 131, 34, 1, 7, 88, 0, 226, 1, 7, 88, 0, 192, 1, 7, 104, 0, 242, 1, 7, 39, 255, 32, 1, 7, 64, 0, 226, 1, 7, 47, 255, 32, 1, 7, 72, 0, 226, 1, 255, 242, 132, 32, 1, 7, 137, 8, 34, 65, 7, 168, 0, 226, 1, 7, 153, 8, 32, 1, 7, 137, 8, 52, 1, 7, 170, 133, 34, 1, 7, 144, 0, 226, 1, 7, 144, 0, 192, 1, 7, 160, 0, 242,


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  5000
[ Success ]      :  0
[ Data ]         :  [0, 1, 0, 3, 11, 161, 1, 255, 225, 8, 32, 1, 0, 3, 10, 161, 1, 255, 225, 8, 34, 65, 9, 240, 0, 226, 1, 10, 40, 0, 192, 1, 9, 247, 255, 32, 1, 255, 225, 8, 34, 1, 10, 0, 0, 226, 1, 10, 0, 0, 192, 1, 7, 127, 255, 32, 1, 9, 168, 0, 226, 1, 7, 179, 54, 32, 1, 10, 80, 0, 226, 1, 8, 91, 55, 32, 1, 10, 88, 0, 226, 1, 9, 171, 56, 32, 1, 10, 96, 0, 226, 1, 9, 3, 57, 32, 1, 10, 104, 0, 226, 1, 10, 81, 8, 32, 1, 255, 104, 0, 2, 1, 10, 89, 8, 32, 1, 255, 112, 0, 2, 1, 10, 97, 8, 32, 1, 255, 120, 0, 2, 1, 10, 105, 8, 32, 1, 255, 128, 0, 2, 1, 1, 0, 5, 0, 8, 28, 0, 28, 1, 0, 4, 0, 8, 29, 8, 1, 0, 5, 0, 8, 30, 0, 0, 1, 0, 4, 0, 8, 31, 0, 1, 0, 6, 0, 8, 32, 0, 0, 0, 1, 0, 6, 0, 8, 33, 0, 0, 0, 1, 0, 5, 0, 8, 34, 0, 0, 1, 0, 5, 0, 8, 35, 0, 0, 1, 0, 5, 0, 8, 36, 128, 0, 1, 0, 5, 0, 8, 37, 0, 0, 1, 0, 5, 0, 8, 38, 0, 0, 1, 0

...


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  9500
[ Success ]      :  0
[ Data ]         :  [81, 236, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 1, 14, 229, 0, 4, 0, 0, 25, 31, 32, 1, 0, 9, 8, 34, 1, 0, 2, 1, 160, 1, 255, 225, 24, 34, 1, 0, 2, 0, 160, 1, 0, 16, 0, 234, 1, 0, 32, 0, 242, 1, 0, 16, 0, 192, 1, 0, 0, 0, 0, 1, 255, 240, 1, 33, 1, 0, 0, 0, 161, 1, 255, 225, 8, 34, 65, 0, 40, 0, 226, 1, 0, 32, 0, 192, 1, 0, 47, 255, 32, 1, 255, 225, 8, 34, 1, 0, 48, 0, 226, 1, 0, 50, 2, 32, 1, 0, 0, 0, 226, 1, 0, 89, 31, 32, 1, 0, 73, 8, 34, 1, 0, 2, 4, 160, 1, 255, 225, 24, 34, 1, 0, 2, 3, 160, 1, 0, 80, 0, 234, 1, 0, 96, 0, 242, 1, 0, 80, 0, 192, 1, 0, 0, 0, 0, 1, 255, 240, 1, 33, 1, 0, 0, 0, 161, 1, 255, 225, 8, 34, 65, 0, 40, 0, 226, 1, 0, 96, 0, 192, 1, 0, 47, 255, 32, 1, 255, 225, 8, 34, 1, 0, 48, 0, 226, 1, 0, 50, 5, 32, 1, 0, 64, 0, 226, 1, 0, 0, 0, 0, 1, 0, 104, 0, 232, 1, 0, 0, 0, 


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  11000
[ Success ]      :  0
[ Data ]         :  [8, 0, 226, 1, 3, 186, 52, 32, 1, 4, 16, 0, 226, 1, 3, 178, 53, 32, 1, 4, 24, 0, 226, 1, 3, 186, 54, 32, 1, 4, 32, 0, 226, 1, 3, 178, 55, 32, 1, 4, 40, 0, 226, 1, 3, 186, 56, 32, 1, 4, 48, 0, 226, 1, 3, 178, 57, 32, 1, 4, 56, 0, 226, 1, 3, 194, 58, 32, 1, 3, 202, 58, 34, 1, 4, 2, 58, 34, 1, 4, 10, 58, 34, 1, 0, 0, 0, 0, 1, 4, 80, 0, 226, 1, 3, 210, 59, 32, 1, 3, 234, 59, 34, 1, 4, 18, 59, 34, 1, 4, 26, 59, 34, 1, 0, 0, 0, 0, 1, 4, 152, 0, 226, 1, 3, 218, 60, 32, 1, 3, 242, 60, 34, 1, 4, 34, 60, 34, 1, 4, 42, 60, 34, 1, 0, 0, 0, 0, 1, 4, 224, 0, 226, 1, 3, 226, 61, 32, 1, 3, 250, 61, 34, 1, 4, 50, 61, 34, 1, 4, 58, 61, 34, 1, 0, 0, 0, 0, 1, 5, 40, 0, 226, 1, 4, 122, 65, 32, 1, 4, 114, 66, 34, 1, 4, 98, 65, 52, 1, 4, 90, 66, 34, 1, 4, 82, 62, 34, 1, 4, 74, 63, 34, 1, 4, 66, 64, 34, 1, 0, 0, 0, 0, 1, 4, 104, 


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  12500
[ Success ]      :  0
[ Data ]         :  [152, 0, 226, 38, 8, 137, 26, 32, 1, 0, 0, 0, 0, 1, 8, 208, 0, 226, 1, 8, 216, 0, 242, 1, 8, 208, 0, 192, 1, 0, 0, 0, 0, 1, 0, 2, 179, 161, 1, 255, 225, 8, 32, 1, 0, 2, 178, 161, 1, 255, 225, 8, 34, 65, 8, 160, 0, 226, 1, 8, 216, 0, 192, 1, 8, 167, 255, 32, 1, 255, 225, 8, 34, 1, 8, 176, 0, 226, 1, 8, 176, 0, 192, 1, 7, 79, 255, 32, 1, 8, 88, 0, 226, 1, 255, 243, 9, 64, 1, 7, 128, 0, 192, 1, 7, 135, 255, 32, 1, 0, 0, 0, 0, 1, 9, 96, 0, 226, 1, 9, 104, 0, 242, 1, 7, 129, 12, 32, 1, 9, 136, 0, 226, 1, 9, 136, 0, 192, 1, 9, 143, 255, 32, 1, 9, 136, 0, 226, 1, 9, 145, 8, 32, 1, 9, 147, 6, 34, 65, 9, 139, 6, 34, 9, 9, 152, 0, 226, 1, 9, 25, 8, 32, 1, 9, 27, 6, 34, 65, 9, 107, 6, 34, 1, 9, 9, 8, 52, 1, 9, 11, 6, 34, 65, 9, 99, 6, 34, 1, 9, 16, 0, 226, 1, 255, 233, 8, 66, 1, 9, 32, 0, 242, 1, 9, 153, 8, 32, 1, 25


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  14000
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


<< ReadResponse >> 
[ Control ]      :  11
[ Total_length ] :  509
[ Chip_address ] :  80
[ Data_length ]  :  500
[ Address ]      :  15500
[ Success ]      :  0
[ Data ]         :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
# clear EEPROM

dsp.eeprom.clear()


[Server receiving: 42 bytes of data.]
[9, 1, 0, 0, 42, 80, 0, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  0
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']

[Server receiving: 10240 bytes of data.]
[9, 1, 0, 0, 42, 80, 0, 32, 0, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 1, 0, 0, 42, 80, 0, 32, 0, 64, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  256
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  288
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0

[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  768
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  800
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  1312
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  1344
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', 


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  1824
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  1856
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', 


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  2528
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  2560
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', 


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  3040
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  3072
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', 


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  3616
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  3648
[ Data ]           :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', 

In [17]:
# load_eeprom_from_file: load data of EEPROM from a binary file.

dsp.control.load_eeprom_from_file(binary_file_name = 'E2Prom.bin')


[Server receiving: 42 bytes of data.]
[9, 1, 0, 0, 42, 80, 0, 32, 0, 0, 1, 0, 5, 0, 8, 28, 0, 88, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 0, 35, 0, 8, 0]

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  0
[ Data ]           :  [1, 0, 5, 0, 8, 28, 0, 88, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 0, 35, 0, 8, 0]
[ Data in Hex ]    :  ['0x1', '0x0', '0x5', '0x0', '0x8', '0x1c', '0x0', '0x58', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x3', '0x1', '0x0', '0x23', '0x0', '0x8', '0x0']

[Server receiving: 6846 bytes of data.]
[9, 1, 0, 0, 42, 80, 0, 32, 0, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 1, 0, 0, 42, 80, 0, 32, 0, 64, 1, 4, 235, 0, 0, 0, 0, 0, 0, 255, 0, 0, 150, 48, 0, 128,


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  224
[ Data ]           :  [0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  256
[ Data ]           :  [0, 0, 0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0',


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  736
[ Data ]           :  [0, 0, 0, 128, 0, 0, 0, 90, 130, 122, 0, 64, 0, 0, 0, 45, 65, 61, 0, 32, 0, 0, 0, 22, 160, 158, 0, 16, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x5a', '0x82', '0x7a', '0x0', '0x40', '0x0', '0x0', '0x0', '0x2d', '0x41', '0x3d', '0x0', '0x20', '0x0', '0x0', '0x0', '0x16', '0xa0', '0x9e', '0x0', '0x10', '0x0', '0x0', '0x0', '0x0']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  768
[ Data ]           :  [19, 7, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 81, 236, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128]
[ Data in Hex ]    :  ['0x13', '0x7', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  1248
[ Data ]           :  [0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 90, 130, 122, 0, 64, 0, 0, 0, 45]
[ Data in Hex ]    :  ['0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x80', '0x0', '0x0', '0x0', '0x5a', '0x82', '0x7a', '0x0', '0x40', '0x0', '0x0', '0x0', '0x2d']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  1280
[ Data ]           :  [65, 61, 0, 32, 0, 0, 0, 22, 160, 158, 0, 16, 0, 0, 0, 0, 19, 7, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 81, 236, 0, 128]
[ Data in Hex ]    :  ['0x41', '0x3d', '0x0', '0x20', '0x0', '0x0', '0x0', '0x

[ Data ]           :  [40, 0, 226, 1, 1, 57, 8, 32, 1, 0, 0, 0, 0, 1, 1, 72, 0, 240, 1, 1, 73, 8, 32, 1, 255, 241, 19, 34, 73, 1, 49, 8]
[ Data in Hex ]    :  ['0x28', '0x0', '0xe2', '0x1', '0x1', '0x39', '0x8', '0x20', '0x1', '0x0', '0x0', '0x0', '0x0', '0x1', '0x1', '0x48', '0x0', '0xf0', '0x1', '0x1', '0x49', '0x8', '0x20', '0x1', '0xff', '0xf1', '0x13', '0x22', '0x49', '0x1', '0x31', '0x8']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  1760
[ Data ]           :  [32, 1, 255, 233, 8, 32, 37, 1, 48, 0, 226, 1, 0, 209, 8, 32, 1, 0, 216, 0, 226, 37, 0, 225, 8, 32, 1, 0, 241, 8, 34, 65]
[ Data in Hex ]    :  ['0x20', '0x1', '0xff', '0xe9', '0x8', '0x20', '0x25', '0x1', '0x30', '0x0', '0xe2', '0x1', '0x0', '0xd1', '0x8', '0x20', '0x1', '0x0', '0xd8', '0x0', '0xe2', '0x25', '0x0', '0xe1', '0x8', '0x20', '0x1', '0x0', '0xf1', '0x8', '0x22', '0x41

...


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  2784
[ Data ]           :  [49, 32, 1, 3, 248, 0, 226, 1, 3, 186, 50, 32, 1, 4, 0, 0, 226, 1, 3, 178, 51, 32, 1, 4, 8, 0, 226, 1, 3, 186, 52, 32]
[ Data in Hex ]    :  ['0x31', '0x20', '0x1', '0x3', '0xf8', '0x0', '0xe2', '0x1', '0x3', '0xba', '0x32', '0x20', '0x1', '0x4', '0x0', '0x0', '0xe2', '0x1', '0x3', '0xb2', '0x33', '0x20', '0x1', '0x4', '0x8', '0x0', '0xe2', '0x1', '0x3', '0xba', '0x34', '0x20']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  2816
[ Data ]           :  [1, 4, 16, 0, 226, 1, 3, 178, 53, 32, 1, 4, 24, 0, 226, 1, 3, 186, 54, 32, 1, 4, 32, 0, 226, 1, 3, 178, 55, 32, 1, 4]
[ Data in Hex ]    :  ['0x1', '0x4', '0x10', '0x0', '0xe2', '0x


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  3296
[ Data ]           :  [1, 5, 234, 95, 34, 1, 5, 242, 94, 34, 1, 5, 250, 93, 34, 1, 6, 2, 92, 34, 1, 6, 8, 0, 226, 1, 4, 249, 8, 32, 1, 6]
[ Data in Hex ]    :  ['0x1', '0x5', '0xea', '0x5f', '0x22', '0x1', '0x5', '0xf2', '0x5e', '0x22', '0x1', '0x5', '0xfa', '0x5d', '0x22', '0x1', '0x6', '0x2', '0x5c', '0x22', '0x1', '0x6', '0x8', '0x0', '0xe2', '0x1', '0x4', '0xf9', '0x8', '0x20', '0x1', '0x6']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  3328
[ Data ]           :  [88, 0, 226, 1, 6, 18, 111, 32, 1, 6, 26, 110, 34, 1, 6, 34, 109, 34, 1, 6, 42, 108, 34, 1, 6, 50, 107, 34, 1, 6, 58, 106]
[ Data in Hex ]    :  ['0x58', '0x0', '0xe2', '0x1', '0x6', '0


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  3808
[ Data ]           :  [58, 174, 34, 9, 8, 72, 0, 226, 1, 7, 201, 8, 32, 1, 7, 202, 174, 34, 65, 8, 26, 174, 34, 1, 7, 185, 8, 52, 1, 7, 186, 174]
[ Data in Hex ]    :  ['0x3a', '0xae', '0x22', '0x9', '0x8', '0x48', '0x0', '0xe2', '0x1', '0x7', '0xc9', '0x8', '0x20', '0x1', '0x7', '0xca', '0xae', '0x22', '0x41', '0x8', '0x1a', '0xae', '0x22', '0x1', '0x7', '0xb9', '0x8', '0x34', '0x1', '0x7', '0xba', '0xae']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  3840
[ Data ]           :  [34, 65, 8, 18, 174, 34, 1, 7, 192, 0, 226, 1, 255, 233, 8, 66, 1, 7, 208, 0, 242, 1, 8, 73, 8, 32, 1, 255, 233, 8, 130, 1]
[ Data in Hex ]    :  ['0x22', '0x41', '0x8', '0x


<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  4352
[ Data ]           :  [0, 2, 178, 161, 1, 255, 225, 8, 34, 65, 8, 160, 0, 226, 1, 8, 216, 0, 192, 1, 8, 167, 255, 32, 1, 255, 225, 8, 34, 1, 8, 176]
[ Data in Hex ]    :  ['0x0', '0x2', '0xb2', '0xa1', '0x1', '0xff', '0xe1', '0x8', '0x22', '0x41', '0x8', '0xa0', '0x0', '0xe2', '0x1', '0x8', '0xd8', '0x0', '0xc0', '0x1', '0x8', '0xa7', '0xff', '0x20', '0x1', '0xff', '0xe1', '0x8', '0x22', '0x1', '0x8', '0xb0']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  4384
[ Data ]           :  [0, 226, 1, 8, 176, 0, 192, 1, 7, 79, 255, 32, 1, 8, 88, 0, 226, 1, 255, 243, 9, 64, 1, 7, 128, 0, 192, 1, 7, 135, 255, 32]
[ Data in Hex ]    :  ['0x0', '0xe2', '0x1', '0

[ Data_length ]    :  32
[ Address ]        :  4832
[ Data ]           :  [10, 11, 50, 34, 1, 9, 184, 0, 226, 1, 255, 233, 8, 66, 1, 9, 200, 0, 242, 1, 10, 65, 8, 32, 1, 255, 233, 8, 130, 1, 255, 233]
[ Data in Hex ]    :  ['0xa', '0xb', '0x32', '0x22', '0x1', '0x9', '0xb8', '0x0', '0xe2', '0x1', '0xff', '0xe9', '0x8', '0x42', '0x1', '0x9', '0xc8', '0x0', '0xf2', '0x1', '0xa', '0x41', '0x8', '0x20', '0x1', '0xff', '0xe9', '0x8', '0x82', '0x1', '0xff', '0xe9']

<< Write >>          
[ Control ]        :  9
[ Safeload ]       :  1
[ Channel_number ] :  0
[ Total_length ]   :  42
[ Chip_address ]   :  80
[ Data_length ]    :  32
[ Address ]        :  4864
[ Data ]           :  [8, 132, 35, 255, 241, 10, 32, 1, 0, 0, 0, 0, 1, 10, 72, 0, 246, 1, 10, 73, 8, 34, 1, 10, 24, 0, 226, 1, 0, 0, 0, 0]
[ Data in Hex ]    :  ['0x8', '0x84', '0x23', '0xff', '0xf1', '0xa', '0x20', '0x1', '0x0', '0x0', '0x0', '0x0', '0x1', '0xa', '0x48', '0x0', '0xf6', '0x1', '0xa', '0x49', '0x8', '0x22', '0x1', '0xa', 

### Close TCP channels

In [20]:
tcpi_client.stop() 
tcpi_server.stop()
t_server.join()


[Client set to stop.]

[Server set to stop ]

[Connection with client ('192.168.0.168', 64410) is closed.]
[Connection with client ('192.168.0.168', 64410) is closed.]

